# Prelimniaries

In [35]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import umap
import hdbscan
from sklearn.preprocessing import StandardScaler

# Data

In [3]:
data = pd.read_parquet('../data/marketing_sample_walmart.parq.gzip')

In [4]:
data.head()

,Uniq Id,Crawl Timestamp,Product Url,Product Name,Description,List Price,Sale Price,Brand,Item Number,Gtin,Package Size,Category,Postal Code,Available
0,51b010b871cde349bd32159a1cc1a15f,2020-01-24 16:08:36 +0000,https://www.walmart.com/ip/Allegiance-Economy-...,Allegiance Economy Dual-scale Digital Thermometer,We aim to show you accurate product informati...,11.11,11.11,Cardinal Health,NaN,707389636164,NaN,Health | Medicine Cabinet | Thermometers | Dig...,NaN,True
1,d6a7f100e44a626a3701804e99236ad6,2020-01-24 15:54:21 +0000,https://www.walmart.com/ip/Kenneth-Cole-Reacti...,Kenneth Cole Reaction Eau De Parfum Spray For ...,We aim to show you accurate product informati...,23.99,23.99,Kenneth Cole,NaN,191565696101,NaN,Premium Beauty | Premium Fragrance | Premium P...,NaN,True
2,99d2b7da7e3e427a942f864937dacd9d,2020-01-24 18:34:28 +0000,https://www.walmart.com/ip/Kid-Tough-Fitness-I...,Kid Tough Fitness Inflatable Free-Standing Pun...,We aim to show you accurate product informati...,30.76,30.76,BONK FIT,563852139.0,855523007070,NaN,Sports & Outdoors | Outdoor Sports | Hunting |...,NaN,True
3,4c76d170c2c6a759cbce812d790a0b88,2020-01-24 11:08:53 +0000,https://www.walmart.com/ip/THE-FIRST-YEARS/167...,THE FIRST YEARS,We aim to show you accurate product informati...,6.99,6.99,The First Years,553299941.0,71463046263,NaN,Baby | Diapering | Baby Wipes,NaN,True
4,8ac95837dc8baa01e504fd8f633ffaf2,2020-03-10 07:37:21 +0000,https://www.walmart.com/ip/4-Pack-MD-USA-Seaml...,4 Pack - MD USA Seamless Toe-Wave-In Mesh Diab...,We aim to show you accurate product informatio...,28.27,28.27,MD USA,NaN,191897514500,NaN,Health | Diabetes Care | Diabetic Socks,NaN,True


In [5]:
data.shape

(30000, 14)

Many of these URLs are invalid (two years old), so I'm going to treat the `Product Name` as the title that would've been retrieved from URL HTML.  Otherwise, we would fetch the titles and/or actual HTML content.

In [6]:
products = data['Product Name'].to_list()

In [7]:
products[:10]

['Allegiance Economy Dual-scale Digital Thermometer',
 'Kenneth Cole Reaction Eau De Parfum Spray For Women 3.40 Oz',
 'Kid Tough Fitness Inflatable Free-Standing Punching Bag + Machine Washable Fabric Cover South Carolina Gamecocks Kids Workout Buddy by Bonk Fit',
 'THE FIRST YEARS',
 '4 Pack - MD USA Seamless Toe-Wave-In Mesh Diabetic Crew Socks, Black, Medium, 1 Pair',
 'Gerber 2nd Foods Apple Baby Food 4 oz. Tubs 2 Count',
 'Kushies Ultra-Lite All-In-One Form-Fitted Washable Cloth Diapers (Blue Whales, Infant)',
 'sunmark Stop Smoking Aid 14 mg Strength Transdermal Patch, 70677003101 - Box of 14',
 'Berkley PowerBait Glitter Chroma-Glow Dough Fishing Bait',
 'Mikasa Rubber Basketball, Intermediate, 28.5']

# Embed Product Names

In [8]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(products, show_progress_bar=True)

Batches: 100%|███████████████████████████████████████████████████████████████████████████████| 938/938 [02:35<00:00,  6.02it/s]


In [9]:
embeddings.shape

(30000, 384)

# Dimensionality Reduction and Clustering

In [10]:
red = umap.UMAP(n_components=int(embeddings.shape[1]*.2), metric='cosine')
red_embed = red.fit_transform(embeddings)

In [11]:
sc = StandardScaler()
red_embed = sc.fit_transform(red_embed)

In [12]:
clust = hdbscan.HDBSCAN(min_cluster_size=15, cluster_selection_epsilon=.25)
clust.fit(red_embed)

HDBSCAN(cluster_selection_epsilon=0.25, min_cluster_size=15)

In [13]:
res = pd.DataFrame({
    'product': products,
    'cluster': clust.labels_
})

In [14]:
res.groupby('cluster').count().sort_values('product', ascending=False)

,product
cluster,
-1,9324
182,850
292,376
233,343
126,319
...,...
298,16
257,15
318,15


# Explore a Example Cluster

In [15]:
mask = res['cluster'] == 277
res[mask]

,product,cluster
345,"Nu-Calgon Drain Solve Liquid Drain Opener, 1 G...",277
2052,OXO Good Grips Silicone Drain Protector for Po...,277
3770,Shampoo Bowl Sink with a Tilt Mechanism Salon ...,277
4861,Kitchen Stainless Steel Sink Strainer Drain St...,277
6453,Premium New Home Cleaning Brushes Drain Sink H...,277
6721,InterDesign Soap Savers Forma Stainless Steel ...,277
6801,"682-685 Tub Stopper with Metal Ring, Rubber - ...",277
7198,Drain Stick SUPERIOR TOOL 3819,277
7278,Core Kitchen Clean Solutions In-Sink Silicone ...,277
7475,TubShroom Revolutionary Hair Catcher Drain Pro...,277


A list makes it easier to read the full product names

In [64]:
[p for p in res.loc[mask, 'product']]

['Nu-Calgon Drain Solve Liquid Drain Opener, 1 Gallon, 4 Per Case',
 'OXO Good Grips Silicone Drain Protector for Pop-Up & Regular Drains',
 'Shampoo Bowl Sink with a Tilt Mechanism Salon Spa Equipment TLC-B36-WT',
 'Kitchen Stainless Steel Sink Strainer Drain Stopper Basket 4.3" Dia',
 'Premium New Home Cleaning Brushes Drain Sink Hair Unclog Sink Clog Remover For Drain Cleaning Tool',
 'InterDesign Soap Savers Forma Stainless Steel Sink Stopper, Silver',
 '682-685 Tub Stopper with Metal Ring, Rubber - Quantity 5',
 'Drain Stick SUPERIOR TOOL 3819',
 'Core Kitchen Clean Solutions In-Sink Silicone Drain Strainer with Lid - Teal',
 'TubShroom Revolutionary Hair Catcher Drain Protector for Tub Drains (No More Clogs) Green',
 'Meigar Silicone Tub Stopper Upgraded Bathtub Drain Plug Cover for Bathroom,Floor Drains,Kitchen and Laundry Universal Use',
 'Tub Stopper Silicone Bathtub Stopper Drain Plug Sinks Hair Stopper Flat Cover',
 '300-305 In-Line Strainer Handle',
 'Westbrass InSinkErator

# Find Cluster Centroids

These can be used to map new data points to appropriate cluster and topic.

In [24]:
centroids = pd.concat([res, pd.DataFrame(red_embed)], axis=1).groupby('cluster').mean()

/var/folders/36/gj41c8c14fnflx3h38bncgqm0000gn/T/ipykernel_97972/4076618808.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  centroids = pd.concat([res, pd.DataFrame(red_embed)], axis=1).groupby('cluster').mean()


In [25]:
centroids.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
cluster,,,,,,,,,,,,,,,,,,,,,
-1,0.054684,0.051837,-0.070168,-0.039248,0.140425,-0.046213,-0.002418,-0.068706,-0.012534,-0.010027,...,-0.030277,-0.034523,-0.053452,-0.091808,0.144425,-0.021548,-0.117913,0.068909,0.100417,0.048953
0,0.477451,0.272541,0.030573,-0.222451,-1.399465,0.096117,0.230955,2.092684,-1.476820,-0.612342,...,-6.601921,1.918673,-1.289168,-6.389969,-2.444455,-0.592760,4.464136,2.442089,-2.314590,-0.457591
1,-0.345042,-0.835367,-1.587616,17.667889,-1.595224,-0.607084,-0.124108,0.002359,-0.415817,1.300347,...,-0.979287,0.581679,0.960818,-0.931504,-0.455036,-1.345986,1.422144,0.070349,-0.171050,-0.469017
2,-0.633364,-0.547898,0.136941,-0.209971,-0.082582,-0.218946,-0.424475,0.146339,0.370710,0.321248,...,2.543015,5.709893,6.238926,-2.668150,4.564625,2.956694,5.399764,-0.157715,-0.173816,-1.270882
3,-0.477172,-0.330979,-1.309273,2.148875,-1.115572,11.963037,10.933246,-0.876655,-0.785724,-0.317436,...,-2.769029,3.037147,-1.373494,1.016186,0.537362,-1.145657,2.432655,0.010051,-1.895911,-1.979177


# Find Similar Clusters

Here we could merge clusters with similarity >= some threshold.

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
similarity = pd.DataFrame(
    np.triu(
        cosine_similarity(centroids, centroids)
    ),
    index=centroids.index,
    columns=centroids.index
)
similarity

cluster,-1,0,1,2,3,4,5,6,7,8,...,376,377,378,379,380,381,382,383,384,385
cluster,,,,,,,,,,,,,,,,,,,,,
-1,1.0,-0.020368,-0.111746,-0.176078,-0.052655,-0.125941,-0.317305,-0.217494,0.253964,-0.381335,...,0.541236,0.609191,-0.331881,-0.240542,-0.144736,-0.166860,-0.347464,-0.286367,-0.224506,-0.402474
0,0.0,1.000000,0.054014,-0.016799,-0.021283,-0.032214,0.034106,0.064630,0.026266,-0.022967,...,0.005884,-0.006050,-0.054285,-0.025810,-0.014805,-0.019085,-0.027529,-0.031374,-0.038555,-0.031502
1,0.0,0.000000,0.999999,-0.036887,0.187239,-0.027979,0.012625,-0.109972,-0.218101,-0.005319,...,-0.008157,-0.003801,-0.050941,-0.156254,-0.182607,-0.108580,-0.047498,-0.095212,-0.087045,-0.071442
2,0.0,0.000000,0.000000,1.000000,-0.076277,-0.051817,0.031714,0.024545,-0.026247,0.032260,...,-0.021872,-0.020991,0.022047,0.093893,0.093377,0.085141,0.058387,0.076679,0.074579,0.090683
3,0.0,0.000000,0.000000,0.000000,1.000000,-0.056376,-0.077320,-0.132918,-0.232614,-0.062292,...,-0.074838,-0.066066,0.053586,-0.107906,-0.177313,-0.016626,0.044889,-0.001209,-0.005886,-0.000454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.897193,0.974231,0.954153,0.873586
382,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.967690,0.966514,0.973166
383,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.983131,0.948973


# Find Most Common Words in Cluster

These would be topics.  We're doing a simple frequency analysis (vice TF-IDF) as we expect documents to be similar, thus aren't interested in words that distinguish them from others in the clusters, but rather words that are common within the cluster.

Intuitively, these results make sense.

In [72]:
from collections import Counter

bow = ' '.join([p for p in res.loc[mask, 'product']]).split(' ')
c = Counter(bow)

In [75]:
c.most_common()[:5]

[('Drain', 43), ('Sink', 23), ('Stopper', 21), ('Hair', 16), ('Strainer', 15)]